# Cuban Human Brain Mapping Project Exploratory Data Analysis

**Update**: *November 24th, 2021*

#### Cuban Neuroscience Center

This dataset encompasses the Cuban Brain Mapping database, featuring data from 279 subjects. It includes both EEG and MRI data (T1, DWI, and Fieldmap).

**Cuban Brain Mapping EEG Protocol**:
- **Total EEG Recording Duration**: 27 minutes maximum.
- **Eyes Closed**: 15 minutes.
- **Eyes Open**: 5 minutes.
- **Hyperventilation**: 3 minutes.
- **Recuperation**: 3 minutes.


In [2]:
import mne 
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

In [3]:
chbmp_filepath = Path('/Users/luiscamargo/Developer/datasets/chbmp')

In [4]:
subject_df = pd.read_csv('/Users/luiscamargo/Developer/datasets/chbmp/CHBMP_EEG_and_MRI/ds_bids_cbm_loris_24_11_21/participants.tsv', delimiter='\t')
subject_df.sex.value_counts()

sex
M    178
F    101
Name: count, dtype: int64

In [5]:
edfs  = sorted([file for file in chbmp_filepath.glob("**/*.edf")])
channel_tsvs = sorted([file for file in chbmp_filepath.glob("**/*task-protmap_channels.tsv")])
events_tsvs = sorted([file for file in chbmp_filepath.glob("**/*task-protmap_events.tsv")])


len(edfs) == len(channel_tsvs) == len(events_tsvs)

True

In [6]:
channel_df = pd.concat((pd.read_csv(tsv, delimiter='\t') for tsv in channel_tsvs))
events_df = pd.concat((pd.read_csv(tsv, delimiter='\t') for tsv in events_tsvs))





In [7]:
channel_df['status'].value_counts()

status
good    20045
bad       312
Name: count, dtype: int64

In [8]:
events_df

,onset,duration,trial_type,value,sample
0,0.005,0,eyes_closed,65,1
1,608.470,0,eyes_open,66,121694
2,621.115,0,eyes_closed,65,124223
3,634.250,0,eyes_open,66,126850
4,652.610,0,eyes_closed,65,130522
...,...,...,...,...,...
27,1091.105,0,hyperventilation_2,68,218221
28,1148.565,0,discontinuity,2,229713
29,1153.935,0,discontinuity,2,230787
30,1168.975,0,hyperventilation_3,69,233795


In [9]:
events_df['duration'] = events_df['onset'].shift(-1) - events_df['onset']


In [10]:
events_df

,onset,duration,trial_type,value,sample
0,0.005,608.465,eyes_closed,65,1
1,608.470,12.645,eyes_open,66,121694
2,621.115,13.135,eyes_closed,65,124223
3,634.250,18.360,eyes_open,66,126850
4,652.610,17.155,eyes_closed,65,130522
...,...,...,...,...,...
27,1091.105,57.460,hyperventilation_2,68,218221
28,1148.565,5.370,discontinuity,2,229713
29,1153.935,15.040,discontinuity,2,230787
30,1168.975,74.970,hyperventilation_3,69,233795


In [11]:
import abc
from mne.io import Raw


class Parser(abc.ABC):
    """
    Abstract base class for objects that interact with MNE in parsing EEG data
    from various systems across physiological datasets.
    """

    @abc.abstractmethod
    def read_eeg(self, filepath: str) -> Raw:
        """
        Abstract method to load EEG data from a file.
        :param filepath: Path to the file
        """
        pass

    @abc.abstractmethod
    def load_eeg(self, raw: Raw) -> Raw:
        """
        Abstract method to perform initial processing on the raw EEG data.
        :param raw: Raw EEG data
        """
        pass

    @abc.abstractmethod
    def reconfigure_eeg_channels(self, raw: Raw) -> Raw:
        """
        Abstract method to reconfigure EEG channels in the raw data.
        :param raw: Raw EEG data
        """
        pass


In [12]:
from mne.io import Raw


class EdfParser(Parser):
    """
    Class for parsing EEG data from EDF files using MNE.
    """

    def read_eeg(self, filepath: str) -> Raw:
        """
        Load EEG data from an EDF file.
        :param filepath: Path to the EDF file
        """
        raw = mne.io.read_raw_edf(filepath, preload=True)
        self.reconfigure_eeg_channels(raw)
        return self.load_eeg(raw)

    def load_eeg(self, raw: Raw) -> Raw:
        """
        Apply the standard 10-20 montage to the raw EEG data.
        :param raw: Raw EEG data
        """
        montage = mne.channels.make_standard_montage("standard_1020")
        raw.set_montage(montage)
        return raw

    def reconfigure_eeg_channels(self, raw: Raw) -> Raw:
        """
        Default implementation of reconfiguring EEG channels.
        This can be overridden in subclasses for specific behavior.
        :param raw: Raw EEG data
        """
        return raw
